# meta - PSO

This modified code implements PSO for the multiple knapsack problem. Here are the key changes and additions:

A new Particle class is introduced to represent each particle in the swarm.
The particle_swarm_optimization function replaces the simulated_annealing function. It initializes a swarm of particles and iteratively updates their positions and velocities.
Each particle's position is represented as a list of integers, where each integer corresponds to the knapsack index for each item. A value equal to the number of knapsacks means the item is not selected.
The particle's velocity is used to update its position, with cognitive and social components influencing the movement.
The global best solution is tracked and updated throughout the iterations.
The main PSO parameters (w, c1, c2) are set to common values, but you may want to tune these for your specific problem.
The main function now calls particle_swarm_optimization instead of simulated_annealing.

This PSO implementation should provide good results for the multiple knapsack problem, potentially exploring the solution space more effectively than the previous Simulated Annealing approach. You may need to adjust the number of particles and iterations to balance between solution quality and computation time.

In [2]:
import random
import math
import pandas as pd
import numpy as np

# Item and Knapsack classes remain the same

# Define the problem function remains the same

# Particle class for PSO
class Particle:
    def __init__(self, items, knapsacks):
        self.position = [random.choice(range(len(knapsacks) + 1)) for _ in items]
        self.velocity = [random.uniform(-1, 1) for _ in items]
        self.best_position = self.position[:]
        self.best_value = self.evaluate(items, knapsacks)

    def evaluate(self, items, knapsacks):
        solution = [Knapsack(k.capacity, k.budget) for k in knapsacks]
        for i, knapsack_index in enumerate(self.position):
            if knapsack_index < len(knapsacks):
                solution[knapsack_index].add_item(items[i])
        return sum(k.total_value() for k in solution) if all(k.is_valid() for k in solution) else 0

    def update_velocity(self, global_best_position, w, c1, c2):
        for i in range(len(self.velocity)):
            r1, r2 = random.random(), random.random()
            cognitive = c1 * r1 * (self.best_position[i] - self.position[i])
            social = c2 * r2 * (global_best_position[i] - self.position[i])
            self.velocity[i] = w * self.velocity[i] + cognitive + social

    def update_position(self, num_knapsacks):
        for i in range(len(self.position)):
            self.position[i] = max(0, min(num_knapsacks, round(self.position[i] + self.velocity[i])))

# PSO algorithm
def particle_swarm_optimization(items, knapsacks, num_particles, num_iterations):
    particles = [Particle(items, knapsacks) for _ in range(num_particles)]
    global_best_position = particles[0].position[:]
    global_best_value = particles[0].best_value

    w, c1, c2 = 0.7, 1.4, 1.4  # PSO parameters

    for _ in range(num_iterations):
        for particle in particles:
            value = particle.evaluate(items, knapsacks)
            if value > particle.best_value:
                particle.best_value = value
                particle.best_position = particle.position[:]
            if value > global_best_value:
                global_best_value = value
                global_best_position = particle.position[:]

        for particle in particles:
            particle.update_velocity(global_best_position, w, c1, c2)
            particle.update_position(len(knapsacks))

    # Convert the best position to a solution
    best_solution = [Knapsack(k.capacity, k.budget) for k in knapsacks]
    for i, knapsack_index in enumerate(global_best_position):
        if knapsack_index < len(knapsacks):
            best_solution[knapsack_index].add_item(items[i])

    return best_solution, global_best_value

# The evaluate_solution function remains the same

def main():
    dataset_csv = 'product_data.csv'
    bin_csv = 'bin.csv'
    items, knapsacks = create_problem(dataset_csv, bin_csv)
    
    num_particles = 50
    num_iterations = 1000

    best_solution, best_value = particle_swarm_optimization(items, knapsacks, num_particles, num_iterations)

    print(f"Best value found: {best_value}")
    for i, knapsack in enumerate(best_solution):
        print(f"\nKnapsack {i+1}:")
        print(f"Total weight: {knapsack.total_weight():.2f}/{knapsack.capacity}")
        print(f"Total price: {knapsack.total_price():.2f}/{knapsack.budget}")
        print("Items:")
        for item in knapsack.items:
            print(f"  {item.category} - {item.name}: weight={item.weight:.2f}, value={item.value:.2f}, price={item.price:.2f}")
    
    print("\nEvaluating solution:")
    total_value, all_constraints_met = evaluate_solution(best_solution, knapsacks)
    print(f"Total value: {total_value:.2f}")
    print(f"All constraints met: {'Yes' if all_constraints_met else 'No'}")

if __name__ == "__main__":
    main()

Best value found: 3852.9400000000005

Knapsack 1:
Total weight: 49.83/50
Total price: 453.59/500
Items:
  Preserved Vegetables - Product G: weight=1.68, value=39.47, price=24.15
  Sweet Pastries - Product C: weight=2.81, value=20.02, price=25.89
  Beverages - Product C: weight=1.36, value=6.02, price=21.62
  Preserved Vegetables - Product C: weight=0.50, value=3.36, price=11.11
  Preserved Vegetables - Product E: weight=0.13, value=92.69, price=3.79
  Sweet Pastries - Product C: weight=3.18, value=56.06, price=28.28
  Beverages - Product E: weight=0.45, value=38.16, price=16.01
  Traditional Snacks - Product J: weight=2.25, value=7.51, price=21.18
  Dried Seafood - Product D: weight=4.21, value=6.71, price=8.56
  Savory Pastries - Product G: weight=0.29, value=8.21, price=17.19
  Canned Fruits - Product J: weight=3.53, value=32.57, price=23.77
  Beverages - Product A: weight=0.30, value=135.60, price=29.24
  Dried Fruits - Product I: weight=2.05, value=54.75, price=9.23
  Sweet Pastrie